In [45]:
%matplotlib inline

import numpy as np
import pandas as pd
import re

import Bio.PDB
import Bio.SeqIO
from Bio import pairwise2
from Bio.SubsMat import MatrixInfo as matlist
from Bio.SeqUtils import seq3

FACTOR=10

In [46]:
# launch pymol

import __main__
__main__.pymol_argv = ['pymol','-Qc'] # Pymol: quiet and no GUI
from time import sleep
import pymol
from pymol import cmd

pymol.finish_launching()

In [47]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [48]:
# paths

hla_fasta = '../data/mhc_alleles/hla/hla_prot.fasta'
pdb_path = '../data/structures'
aligned_pdb_path = '../sandbox/aligned_pdb'
corrected_pdb_path = '../sandbox/corrected_pdb'

In [49]:
# tables

annotation = pd.read_csv('../sandbox/detailed_mhc', sep='\t')
alleles_vs_pdb = pd.read_csv('../sandbox/alleles_vs_pdb', sep='\t', index_col=0, low_memory=False)
peptides_vs_pdb = pd.read_csv('../sandbox/peptides_vs_pdb', sep='\t', index_col=0, low_memory=False)
ligand_assays = pd.read_csv('../data/epitopes/mhc_ligand_full.csv', skiprows=1, low_memory=False)

In [50]:
# table conversion

ligand_assays = ligand_assays[ligand_assays['MHC allele class'] == 'I']
affinity = ligand_assays[['Description', 'Allele Name', 'Qualitative Measure', 'Quantitative measurement']]
affinity['Length'] = [len(x) for x in affinity.Description]
affinity = affinity[(affinity.Length < 12) & (affinity.Length > 7)]
affinity = affinity[affinity['Allele Name'].str.contains('HLA-[ABC]\*\S+$', regex=True)]
affinity['Allele Name'] = [x[4:] for x in affinity['Allele Name']]

iclass = annotation[annotation.mhc_class == 'I']
iclass = iclass[iclass.a_chain_allele.str.contains('[ABC]\*', regex=True)]
iclass.a_chain_allele = [':'.join(x.split(':')[:2]) for x in iclass.a_chain_allele]
iclass = iclass.reset_index(drop=True)

In [51]:
# place mhc to the specific position
def localize_pdb(name, amhc, bmhc, pept, plen):
    # define peptide terminal CAs
    parser = Bio.PDB.PDBParser(QUIET=True)
    ppb = Bio.PDB.CaPPBuilder() 
    struct = parser.get_structure(id=name, file=pdb_path+'/'+name+'.pdb')[0]
    reslist = ppb.build_peptides(struct[pept])[0]
    firstCA = reslist[0].get_id()[1]
    lastCA = reslist[-1].get_id()[1]
    
    assert(lastCA-firstCA+1 == plen)
    
    # translate complex by -startCA
    cmd.select('sele', '%s//%s/%i/CA %s//%s/%i/CA' % (name, pept, firstCA, name, pept, lastCA))
    start, end = cmd.get_coords('sele', 1)
    cmd.translate(list(-start), name)
    
    # rotate complex around (0, 0, 0), so that endCA.y = 0
    start, end = cmd.get_coords('sele', 1)
    angle = np.arctan(end[1]/end[0])
    if end[0] < 0.0:
        angle = np.pi + angle
    angle = -angle*180.0/np.pi
    cmd.rotate('z', angle, name, origin=[0.0, 0.0, 0.0])
    
    # superpose peptide line on x axis
    start, end = cmd.get_coords('sele', 1)
    angle = np.arctan(end[2]/end[0])
    if end[0] < 0.0:
        angle = np.pi + angle
    angle = angle*180.0/np.pi
    print(cmd.rotate('y', angle, name, origin=[0.0, 0.0, 0.0]))
    
    # shift along x axis
    start, end = cmd.get_coords('sele', 1)
    print(cmd.translate([-(start[0]+end[0])/2.0, 0.0, 0.0], name))
    
    # superpose com on the xz plane
    com = cmd.centerofmass('%s//%s// %s//%s//' % (name, amhc, name, bmhc))
    angle = np.arctan(com[2]/com[1])
    if com[1] < 0.0:
        angle = np.pi + angle
    angle = -(angle)*180.0/np.pi
    print(cmd.rotate('x', angle, name, origin=[0.0, 0.0, 0.0]))
    
    # verify the result
    com = cmd.centerofmass('%s//%s// %s//%s//' % (name, amhc, name, bmhc))
    start, end = cmd.get_coords('sele', 1)
    assert(com[2] < 0.001)
    assert(start[2] < 0.001 and start[1] < 0.001)
    assert(end[2] < 0.001 and end[1] < 0.001)
    
    #cmd.delete('!(%s//%s// %s//%s// %s//%s//)' % (name, mhca, name, mhcb, name, pept))
    
    #cmd.save('../sandbox/positioned_pdb/%s_posed.pdb' % name, 
    #         '%s//%s// %s//%s// %s//%s//' % (name, mhca, name, mhcb, name, pept))
    
    sleep(0.5)

# name should be preloaded to pymol with cmd.load()    
def localize_pdb_wrapper(name, table):
    line = table[table.pdb == name].iloc[0, :]
    mhca = line['a_chain_id']
    mhcb = line['b_chain_id']
    pept = line['antigen_id']
    plen = int(line['antigen_len'])

    localize_pdb(name, mhca, mhcb, pept, plen)
    
# name should be preloaded to pymol with cmd.load()    
def clean(name, table):
    line = table[table.pdb == name].iloc[0, :]
    mhca = line['a_chain_id']
    mhcb = line['b_chain_id']
    pept = line['antigen_id']
    
    cmd.delete('!(%s//%s// %s//%s// %s//%s//)' % (name, mhca, name, mhcb, name, pept))
    
# reference and mobile should be preloaded to pymol with cmd.load()
def align_mob2ref(mob_name, ref_name, table):
    line = table[table.pdb == ref_name].iloc[0, :]
    ref_mhca = line['a_chain_id']
    ref_mhcb = line['b_chain_id']
    ref_pept = line['antigen_id']
    
    line = table[table.pdb == mob_name].iloc[0, :]
    mob_mhca = line['a_chain_id']
    mob_mhcb = line['b_chain_id']
    mob_pept = line['antigen_id']

    cmd.align('%s//%s// %s//%s//' % (mob_name, mob_mhca, mob_name, mob_pept), 
              '%s//%s// %s//%s//' % (ref_name, ref_mhca, ref_name, ref_pept))
    
# name should be preloaded to pymol with cmd.load()
def save2pdb(name, table, path):
    line = table[table.pdb == name].iloc[0, :]
    mhca = line['a_chain_id']
    mhcb = line['b_chain_id']
    pept = line['antigen_id']
    
    cmd.save(path, '%s//%s// %s//%s// %s//%s//' % (name, mhca, name, mhcb, name, pept)) 

In [52]:
def align_all_pdbs(table):
    ref=table.loc[0, 'pdb']

    cmd.reinitialize()
    cmd.load(pdb_path+'/'+ref+'.pdb', ref)
    clean(ref, table)
    localize_pdb_wrapper(ref, table)

    for i in range(1, table.shape[0]):
        name=table.loc[i, 'pdb']
        cmd.load(pdb_path+'/'+name+'.pdb', name)
        clean(name, table)
        align_mob2ref(name, ref, table)
        save2pdb(name, table, '%s/%s_aligned.pdb' % (aligned_pdb_path, name))
        cmd.delete(name)

    save2pdb(ref, table, '%s/%s_aligned.pdb' % (aligned_pdb_path, ref))
    cmd.delete(ref)
        
#align_all_pdbs(iclass)

In [53]:
iclass.head()

,pdb,mhc_class,host,a_chain_id,a_chain_allele,b_chain_id,b_chain_allele,antigen_id,antigen_seq,antigen_len,antigen_name,antigen_organism,comments
0,3i6l,I,Human,D,A*24:02,E,B2M,F,QFKDNVILL,9,Nucleoprotein,SARS coronavirus Tor2,NaN
1,5swq,I,Human,A,A*02:01,B,B2M,C,CVNGSCFTV,9,neuraminidase,Influenza A virus,NaN
2,3x14,I,Human,A,B*08:02,B,B2M,C,FLRGRAYGL,9,nuclear antigen EBNA-3,Human herpesvirus 4,NaN
3,4jqx,I,Human,A,B*44:03,C,B2M,B,EECDSELEIKRY,12,Trans-activator protein BZLF1,Human herpesvirus 4,NaN
4,5c0d,I,Human,A,A*02:01,B,B2M,C,AQWGPDPAAA,10,NaN,NaN,NaN


In [54]:
def set_values(matrix, atom_desc, left_bottom, right_up, bin_size):
    aas = ['GLY', 'ALA', 'ILE', 'LEU', 
           'PRO', 'SER', 'THR', 'CYS', 
           'MET', 'ASP', 'ASN', 'GLU', 
           'GLN', 'LYS', 'ARG', 'HIS', 
           'PHE', 'TYR', 'TRP']
    
    x = atom_desc[5]
    y = atom_desc[6]
    z = atom_desc[7]
    
    # check if the atom is inside of the box
    if (x-left_bottom[0] < 0.0 or y-left_bottom[1] < 0.0 or z-left_bottom[2] < 0.0):
        return 0
    
    if (x-right_up[0] >= 0.0 or y-right_up[1] >= 0.0 or z-right_up[2] >= 0.0):
        return 0
    
    # compute atom index
    x_bin = np.floor((x-left_bottom[0])/bin_size)
    y_bin = np.floor((y-left_bottom[1])/bin_size)
    z_bin = np.floor((z-left_bottom[2])/bin_size)
    
    #if y_bin == 20:
    #        print(y-left_bottom[1])
    
    # increase values in matrix corresponding to the atom
    if atom_desc[0] != 'ATOM':
        matrix['X'][x_bin, y_bin, z_bin] += 1.0
        matrix['XXX'][x_bin, y_bin, z_bin] += 1.0
        return 1
    
    if atom_desc[1] == 'CA':
        matrix['CA'][x_bin, y_bin, z_bin] += 1.0
        
    if atom_desc[2] in ['C', 'N', 'O', 'S']:
        matrix[atom_desc[2]][x_bin, y_bin, z_bin] += 1.0
    else:
        matrix['X'][x_bin, y_bin, z_bin] += 1.0
    
    if atom_desc[3] in aas:
        matrix[atom_desc[3]][x_bin, y_bin, z_bin] += 1.0
    else:
        matrix['XXX'][x_bin, y_bin, z_bin] += 1.0
        
    if atom_desc[4] == 'H':
        matrix['HELIX'][x_bin, y_bin, z_bin] += 1.0
        
    if atom_desc[4] == 'S':
        matrix['STRAND'][x_bin, y_bin, z_bin] += 1.0
        
    return 2

# convert complex to 4d matrix
# name should be preloaded to pymol with cmd.load()
def mhc2matrix(name, left_bottom=(-30.0, -10.0, -10.0), right_up=(30.0, 10.0, 10.0), bin_size=1.0):
    x_bins = int((right_up[0]-left_bottom[0])/bin_size)
    y_bins = int((right_up[1]-left_bottom[1])/bin_size)
    z_bins = int((right_up[2]-left_bottom[2])/bin_size)
    
    layer_names = ['CA', 'C', 'N', 'O', 'S', 'X', 
                   'GLY', 'ALA', 'ILE', 'LEU', 
                   'PRO', 'SER', 'THR', 'CYS', 
                   'MET', 'ASP', 'ASN', 'GLU', 
                   'GLN', 'LYS', 'ARG', 'HIS', 
                   'PHE', 'TYR', 'TRP', 'XXX', 
                   'HELIX', 'STRAND']
    
    filter_num = len(layer_names)
    m = np.zeros((x_bins, y_bins, z_bins), dtype=zip(layer_names, [np.float32]*len(layer_names)))
    
    atoms = []
    myspace = {'atoms': atoms}
    cmd.iterate_state(0, name, 'atoms.append([type, name, elem, resn, ss, x, y, z])', space=myspace)
    #print(atoms)
    
    for atom_desc in atoms: #cmd.iterate_state(0, name+'//C//', 'type, name, elem, resn, ss, x, y, z'):
        set_values(m, atom_desc, left_bottom, right_up, bin_size)
        
    return np.array([m[name] for name in layer_names])

In [55]:
# select terminal peptide CA and all atoms in range of 15 A from the peptide
def select_atoms(name, pept, plen):
    parser = Bio.PDB.PDBParser(QUIET=True)
    ppb = Bio.PDB.CaPPBuilder() 
    struct = parser.get_structure(id=name, file=corrected_pdb_path+'/'+name+'.pdb')[0]
    reslist = ppb.build_peptides(struct[pept])[0]
    firstCA = reslist[0].get_id()[1]
    lastCA = reslist[-1].get_id()[1]
    
    assert(lastCA-firstCA+1 == plen)
    
    cmd.select('fixed', '%s//%s/%i/CA %s//%s/%i/CA byres %s be. 15 of %s//%s//' % (name, pept, firstCA, 
                                                                                   name, pept, lastCA, 
                                                                                   name, name, pept))
    idx = []
    myspace = {'idx': idx}
    cmd.iterate_state(0, name, 'idx.append(ID)', space=myspace)  
    
    return idx

In [56]:
def get_best_pdbs(allele, pepseq, mhc2pdb, pep2pdb):
    chunk = mhc2pdb.loc[mhc2pdb.index.str.startswith(allele), :]
    best_pdbs = (chunk.apply(min) + pep2pdb.loc[pepseq, :]/10).sort_values()
    return best_pdbs

In [71]:
def ungapped_align(a, b, matrix, peptide=False):
    score = -np.inf
    pos = []
    
    if not peptide:
        for i in range(len(a) + len(b)):
            a_beg, a_end = (max(0, i-len(b)), min(i, len(a)))
            b_beg, b_end = (max(0, len(b)-i), min(len(b), len(b)+len(a)-i))

            loc_score = sum([matrix[(k, t)] for k, t in zip(a[a_beg:a_end], b[b_beg:b_end])])
            if loc_score >= score:
                    score = loc_score
                    pos = [a_beg, b_beg]
    else:
        pos = [0, 0]
    
    a_aligned = a
    b_aligned = b
    
    if (pos[0] > pos[1]):
        b_aligned = '-'*(pos[0]-pos[1])+b_aligned
    else: 
        a_aligned = '-'*(pos[1]-pos[0])+a_aligned
        
    if (len(a_aligned) > len(b_aligned)):
        b_aligned += '-'*(len(a_aligned) - len(b_aligned))
    else:
        a_aligned += '-'*(len(b_aligned) - len(a_aligned))
    
    return a_aligned, b_aligned

# s1 - mutated sequence
# s2 - reference sequence
def align(s1, s2, peptide=False):
    matrix = {}
    for key, val in matlist.blosum62.iteritems(): # do it outside the function
        matrix.update({(key[1], key[0]): val})
        matrix.update({(key[0], key[1]): val})
    
    top_aln = ungapped_align(s1, s2, matrix, peptide)

    print("\tOld: "+top_aln[0])
    print("\tNew: "+top_aln[1])
    
    # check that there is no gaps inside the alignment
    assert(re.search('[A-Z]-+[A-Z]', top_aln[0]) == None)
    assert(re.search('[A-Z]-+[A-Z]', top_aln[1]) == None)
    
    mutations = []
    gap = re.match('(-+)', top_aln[0])
    shift = 0
    if gap:
        shift = gap.end(1) - gap.start(1)
        
    for i, a1, a2 in zip(list(range(-shift, len(top_aln[0])-shift)), top_aln[0], top_aln[1]):
        if a1 != '-' and a2 != '-':
            if a1 != a2:
                mutations.append((i, a1, a2))
    return mutations

'''def align(s1, s2):
    matrix = matlist.blosum62
    gap_open = -100.0
    gap_extend = -1.0
    alns = pairwise2.align.globalds(s1, s2, matrix, gap_open, gap_extend)
    top_aln = alns[0]
    
    
    print(top_aln[0])
    print(top_aln[1])
    
    # check that there is no gaps inside the alignment
    assert(re.search('[A-Z]-+[A-Z]', top_aln[0]) == None)
    assert(re.search('[A-Z]-+[A-Z]', top_aln[1]) == None)
    
    mutations = []
    gap = re.match('(-+)', top_aln[0])
    shift = 0
    if gap:
        shift = gap.end(1) - gap.start(1)
        
    for i, a1, a2 in zip(list(range(-shift, len(top_aln[0])-shift)), top_aln[0], top_aln[1]):
        if a1 != '-' and a2 != '-':
            if a1 != a2:
                mutations.append((i, a1, a2))
    return mutations'''

# mutate pdb structure in accordance to input sequences
# here we assume that all chains are continuous: CHECK THIS
def mutate(pdb, newseq, newpep, table):
    parser = Bio.PDB.PDBParser(QUIET=True)
    ppb=Bio.PDB.PPBuilder()
    
    name = pdb; 
    amhc = table[table.pdb == pdb].iloc[0, 3]
    pept = table[table.pdb == pdb].iloc[0, 7]
    b2m = table[table.pdb == pdb].iloc[0, 5]
    
    struct = parser.get_structure(id=name, file=pdb_path+'/'+name+'.pdb')[0]
    oldseq = ''.join([str(x.get_sequence()) for x in ppb.build_peptides(struct[amhc])])
    oldpep = table[table.pdb == pdb].iloc[0, 8]
    
    assert(len(oldpep) == len(newpep))
    print("\n======= MHC business =======\n")
    mhc_mutations = align(oldseq, newseq)
    print("\nMHC mutations: %i\n" % len(mhc_mutations))
    
    #assert(len(mhc_mutations) < 30)
    
    print("\n===== Peptide business =====\n")
    pep_mutations = align(oldpep, newpep, True)
    print("\nPeptide mutations: %i\n\n" % len(pep_mutations))
    
    print("Mutagenesis: \n")
    # remove alternative atom locations
    cmd.remove("not alt ''+A")
    cmd.alter('all', "alt=''")
    
    # mutate
    cmd.wizard('mutagenesis')
    cmd.refresh_wizard()

    first_aa_id = ppb.build_peptides(struct[amhc])[0][0].get_id()[1]
    for i, old, new in mhc_mutations:
        print(i, old, new)
        cmd.get_wizard().do_select(amhc+'/'+str(i+first_aa_id)+'/')
        cmd.get_wizard().set_mode(seq3(new).upper())
        cmd.get_wizard().apply()
        
    first_aa_id = ppb.build_peptides(struct[pept])[0][0].get_id()[1]
    for i, old, new in pep_mutations:
        print(i, old, new)
        cmd.get_wizard().do_select(pept+'/'+str(i+first_aa_id)+'/')
        cmd.get_wizard().set_mode(seq3(new).upper())
        cmd.get_wizard().apply()
        
    cmd.set_wizard()
    
    #sleep(0.5)
    return (len(mhc_mutations), len(pep_mutations)) # for tests TEMPORARY

In [70]:
#allele = iclass[iclass.pdb == '5brz']['a_chain_allele'].iloc[0]
#pepseq = iclass[iclass.pdb == '5brz']['antigen_seq'].iloc[0]
#chunk = alleles_vs_pdb.loc[alleles_vs_pdb.index.str.startswith(allele), :]
#best_pdbs = (chunk.apply(min) + peptides_vs_pdb.loc[pepseq, :]/FACTOR).sort_values()
#chunk['3bo8']

In [67]:
alleledict = Bio.SeqIO.to_dict(Bio.SeqIO.parse(hla_fasta, 'fasta'))
for key in alleledict.keys():
    alleledict[key.split(';')[1]] = alleledict.pop(key)

# wrapper for mutate(..)
def find_and_mutate(pepseq, allele, alleledict, pep2pdb, mhc2pdb, pdb_table, first_best=10, ref=None):
    if ref:
        print("\nChanging to "+ref+"\n")
        allele = pdb_table[pdb_table.pdb == ref]['a_chain_allele'].iloc[0]
        pepseq = pdb_table[pdb_table.pdb == ref]['antigen_seq'].iloc[0]
    
    chunk = mhc2pdb.loc[mhc2pdb.index.str.startswith(allele), :]
    best_pdbs = (chunk.apply(min) + pep2pdb.loc[pepseq, :]/FACTOR).sort_values()
    
    ############# for tests TEMPORARY
    report = open('../sandbox/mutated/report', 'a')
    report.write('name\tpdb\ta_chain\told_allele\tnew_allele\tmhc_mutations\tpeptide_chain\told_peptide\tnew_peptide\tpeptide_mutations\tcompatibility_score\trmsd\n')
    #############
    
    counter = 0
    for i in xrange(len(best_pdbs)):
        try:
            print("\n=============================================")    
            
            best_pdb = best_pdbs.index[i]
            comp_score = best_pdbs[best_pdb]
            
            if np.isnan(comp_score):
                print("No compatible structures found")
                break
            
            print("Operating on "+best_pdb+"\n")    
                
            allele_list = chunk.loc[:, best_pdb]
            full_allele = allele_list[allele_list == min(allele_list)].index[0]
            print("New allele: "+full_allele)
            
            alleleseq = str(alleledict[full_allele].seq)
            print('Compability score: %f' % comp_score)
            
            ####################### for tests TEMPORARY
            rmsd = peptides_rmsd(corrected_pdb_path+'/'+ref+'.pdb', 
                                 pdb_table[pdb_table.pdb == ref]['antigen_id'].iloc[0], 
                                 corrected_pdb_path+'/'+best_pdb+'.pdb', 
                                 pdb_table[pdb_table.pdb == best_pdb]['antigen_id'].iloc[0])
            
            print("Peptide RMSD: "+str(rmsd))
            
            #if (rmsd <= 1.3 or rmsd > 1.6 or allele == pdb_table[pdb_table.pdb == best_pdb]['a_chain_allele'].iloc[0]):
            #    continue
            
            cmd.save('../sandbox/mutated/ref_%s.pdb' % ref, 'obj1')
            #######################
            
            cmd.reinitialize()
            cmd.load(corrected_pdb_path+'/'+best_pdb+'.pdb', best_pdb)
            #cmd.save('../sandbox/mutated/original%i.pdb' % counter, best_pdb)
            
            mhc_mut_num, pep_mut_num = mutate(best_pdb, alleleseq, pepseq, pdb_table)
            line = pdb_table[pdb_table.pdb == best_pdb].iloc[0, :]
            
            ######################## for tests TEMPORARY
            report.write('%s\t%s\t%s\t%s\t%s\t%i\t%s\t%s\t%s\t%i\t%.1f\t%.2f\n' % 
                         ('mutated%i.pdb' % counter, best_pdb, line.a_chain_id, line.a_chain_allele, allele, mhc_mut_num,
                          line.antigen_id, line.antigen_seq, pepseq, pep_mut_num, comp_score, rmsd))
            ########################
            
            cmd.save('../sandbox/mutated/mutated%i.pdb' % counter, best_pdb)
            
            # write fixed atoms for minimization protocol
            idx = select_atoms(best_pdb, line.antigen_id, line.antigen_len)
            with open('../sandbox/mutated/fixed_atoms%i' % counter, 'w') as f:
                for ind in idx:
                    f.write("%i\n" % ind)
            
            cmd.delete(best_pdb)
            
            # HERE WE SHOULD HAVE MINIMIZER INVOKED
            
        except AssertionError:
            print('Assertion error: %s, %s, %s' % (pepseq, allele, best_pdb))
            continue
    
        except pymol.CmdException:
            print('Pymol error: %s, %s, %s' % (pepseq, allele, best_pdb))
            continue
        else:
            if counter == first_best-1:
                report.close()
                break
                
            else:
                counter += 1
    
    return

In [69]:
find_and_mutate(None, None, alleledict, peptides_vs_pdb, alleles_vs_pdb, iclass, first_best=10, ref='5brz')

In [14]:
# allele dictionary
alleledict = Bio.SeqIO.to_dict(Bio.SeqIO.parse(hla_fasta, 'fasta'))
for key in alleledict.keys():
    alleledict[key.split(';')[1]] = alleledict.pop(key)
    
idx = 100000
pepseq = affinity.iloc[idx, 0]
allele = affinity.iloc[idx, 1]
#ms = find_and_mutate(pepseq, allele, alleledict, peptides_vs_pdb, alleles_vs_pdb, iclass)

In [85]:
iclass[iclass.a_chain_allele == 'A*11:01']
#iclass[iclass.pdb == '1a1m']

,pdb,mhc_class,host,a_chain_id,a_chain_allele,b_chain_id,b_chain_allele,antigen_id,antigen_seq,antigen_len,antigen_name,antigen_organism,comments
42,1q94,I,Human,D,A*11:01,E,B2M,F,AIFQSSMTK,9,Pol polyprotein,Human immunodeficiency virus 1,NaN
44,4mj6,I,Human,A,A*11:01,B,B2M,C,TMVMELIRMIK,11,NaN,NaN,NaN
60,4n8v,I,Human,D,A*11:01,E,B2M,F,MLIYSMWGK,9,Virion envelope protein A14,Vaccinia virus Copenhagen,NaN
97,4mj5,I,Human,A,A*11:01,B,B2M,C,TIAMELIRMIK,11,NaN,NaN,NaN
162,1qvo,I,Human,D,A*11:01,E,B2M,F,QVPLRPMTYK,10,Protein Nef,Human immunodeficiency virus type 1 (JRCSF ISO...,Imported from the TopBank database.
205,2hn7,I,Human,A,A*11:01,B,B2M,C,AIMPARFYPK,10,NaN,NaN,This peptide is an analog of the naturally occ...
247,1x7q,I,Human,A,A*11:01,B,B2M,C,KTFPPTEPK,9,Nucleoprotein,SARS coronavirus Tor2,NaN


In [438]:
iclass.groupby('a_chain_allele').count()

,pdb,mhc_class,host,a_chain_id,b_chain_id,b_chain_allele,antigen_id,antigen_seq,antigen_len,antigen_name,antigen_organism,comments
a_chain_allele,,,,,,,,,,,,
A*01:01,6,6,6,6,6,6,6,6,6,6,6,0
A*02:01,186,186,186,186,186,186,186,186,186,101,101,19
A*02:03,1,1,1,1,1,1,1,1,1,1,1,1
A*02:06,1,1,1,1,1,1,1,1,1,1,1,1
A*02:07,1,1,1,1,1,1,1,1,1,1,1,1
A*02:30,1,1,1,1,1,1,1,1,1,1,1,0
A*02:562,1,1,1,1,1,1,1,1,1,0,0,0
A*02:66,1,1,1,1,1,1,1,1,1,0,0,1
A*03:01,3,3,3,3,3,3,3,3,3,3,3,1


In [22]:
def peptides_rmsd(path1, chain1, path2, chain2):
    cmd.reinitialize()
    cmd.load(path1, 'obj1')
    cmd.load(path2, 'obj2')
    
    res1 = []
    space = {'ar' : res1}
    cmd.iterate_state(0, 'obj1//%s//CA' % chain1, 'ar.append((resi, resn))', space=space)
    cmd.alter('obj1//%s//' % chain1, 'segi=""')
    cmd.alter('obj1//%s//' % chain1, 'chain="XXX"')
    #print(res1)
    
    # change identifiers of obj2 CA to make them similar to obj1
    resi2 = []
    space = {'ar' : resi2}
    cmd.iterate_state(0, 'obj2//%s//CA' % chain2, 'ar.append(resi)', space=space)
    cmd.alter('obj2//%s//' % chain2, 'segi=""')
    cmd.alter('obj2//%s//' % chain2, 'chain="XXX"')
    for idx, newidx in zip(resi2, res1):
        cmd.alter('obj2//XXX/%s/' % idx, 'resi="%s"' % newidx[0])
        cmd.alter('obj2//XXX/%s/' % idx, 'resn="%s"' % newidx[1])
        
    rmsd = cmd.rms('obj1//XXX//CA', 'obj2//XXX//CA')

    return rmsd
    
def rmsd_list(name, pdbtable, pep2pdb, mhc2pdb, maxnum=20):
    # read info for molecule 'name'
    line1 = pdbtable[pdbtable.pdb == name].iloc[0, :]
    path1 = corrected_pdb_path+'/'+name+'.pdb'
    chain1 = line1.antigen_id
    allele1 = line1.a_chain_allele
    pepseq1 = line1.antigen_seq
    
    # check that peptide is present in pep2pdb table
    if pepseq1 not in pep2pdb.index:
        print(name+': bad peptide')
        return {name: [-1]*maxnum}
    
    # discard similar alleles
    nonsimilar = pdbtable[pdbtable.a_chain_allele != allele1].pdb
    
    res = []
    chunk = mhc2pdb.loc[mhc2pdb.index.str.startswith(allele1), :]
    #best_pdbs = (chunk.apply(min) + pep2pdb.loc[pepseq1, :]/FACTOR)[nonsimilar].sort_values()
    best_pdbs = (chunk.apply(min) + pep2pdb.loc[pepseq1, :]/FACTOR).sort_values()
    for pdb in best_pdbs.index[:maxnum]:
        if (best_pdbs[pdb] < np.inf):
            line2 = pdbtable[pdbtable.pdb == pdb].iloc[0, :]
            path2 = corrected_pdb_path+'/'+pdb+'.pdb'
            chain2 = line2.antigen_id
            #print(name+' vs '+pdb+': '+str(peptides_rmsd(path1, chain1, path2, chain2)))
            res.append(peptides_rmsd(path1, chain1, path2, chain2))
        else:
            #print(name+' vs '+pdb+': do not match')
            res.append(np.nan)
    return {name: res}
    
#rmsd_dict = {}
#for name in iclass.pdb:
#    rmsd_dict.update(rmsd_list(name, iclass, peptides_vs_pdb, alleles_vs_pdb))

In [126]:
sum((pd.DataFrame(rmsd_dict).loc[:10, :].replace(-1.0, 10) < 1.0).apply(any))/float(len(rmsd_dict))

0.77944862155388472

In [38]:
rmsd_table = pd.read_csv('../sandbox/peptide_rmsd_nonred', sep='\t')
rmsd_table

,1a1m,1a1n,1a1o,1a9e,1agb,1agc,1agd,1age,1agf,1akj,...,5hgd,5hgh,5hhn,5hho,5hhp,5hhq,5inc,5ind,5iro,5swq
0,1.139542,1.132532,1.327055,0.326978,2.086508e+00,2.166762e+00,2.155766e+00,2.121538e+00,2.107955e+00,1.337969,...,2.536710e+00,2.524906e+00,0.989251,1.538476,1.343414,1.096058,-1,1.675522,1.148401,4.941041e-02
1,1.206494,1.034485,1.320663,1.622864,5.807886e-01,4.132785e-01,4.071409e-01,3.949687e-01,3.758281e-01,1.135945,...,6.040261e-11,6.040261e-11,1.254031,0.983763,1.053181,1.222808,-1,1.685045,1.143062,2.114923e-01
2,1.092499,1.019194,0.460382,1.350060,6.051277e-01,6.595728e-01,5.924562e-01,5.908124e-01,6.035284e-01,1.207939,...,2.629005e+00,2.610812e+00,0.944017,0.863628,1.014962,1.062107,-1,1.679242,0.583344,0.000000e+00
3,1.697405,2.097007,0.454211,1.342483,4.747235e-01,3.590615e-01,3.531187e-01,3.444836e-01,2.776660e-01,1.039592,...,2.575205e+00,2.561959e+00,0.920729,1.205438,1.103532,1.081025,-1,1.651795,1.109387,0.000000e+00
4,1.703075,1.028739,1.350060,0.957108,1.062962e+00,9.023880e-01,9.070487e-01,8.877627e-01,1.155452e+00,0.726095,...,2.659719e+00,2.648761e+00,0.368661,0.522089,0.472819,0.492953,-1,1.409153,1.224504,0.000000e+00
5,1.622864,1.241411,0.476393,1.255134,8.332888e-01,7.382293e-01,7.284047e-01,7.574609e-01,7.270477e-01,0.807407,...,2.566694e+00,2.555122e+00,0.560062,0.866106,0.684999,0.668898,-1,1.680090,0.581473,2.898664e-01
6,1.218730,0.887763,0.463680,1.260010,5.329272e-01,4.068665e-01,4.070600e-01,4.201115e-01,3.662108e-01,0.618642,...,2.271165e+00,2.283606e+00,0.271339,0.673869,0.409925,0.269813,-1,1.588419,1.208005,1.717771e-01
7,1.386504,0.902388,0.468978,1.429574,6.367482e-01,6.869991e-01,6.853245e-01,6.887801e-01,8.408865e-01,0.951957,...,2.600358e+00,2.586132e+00,0.539762,0.917931,0.742238,0.640306,-1,1.447534,0.574689,0.000000e+00
8,1.171789,1.062962,0.454285,0.890916,6.040261e-11,6.040261e-11,6.040261e-11,6.040261e-11,6.040261e-11,1.680874,...,2.603833e+00,2.590298e+00,1.594993,1.893710,1.617371,1.567389,-1,1.604360,1.212600,6.040261e-11
9,1.180262,0.907049,0.800505,0.468812,5.766785e-01,4.084007e-01,3.551152e-01,3.821214e-01,5.653157e-01,0.971857,...,2.070001e+00,2.053192e+00,0.721134,1.150041,0.867111,0.665036,-1,1.647371,0.541458,3.308478e-01


In [167]:
t1 = pd.read_csv('/home/mikhail/Projects/SAXS/libsaxs/examples/benchmark/Real_new/piper/2CFU/prms/rot70k.0.0.4.prm', 
                 sep='\s+', header=None)
t2 = pd.read_csv('/home/mikhail/Projects/SAXS/libsaxs/examples/benchmark/Real_new/piper/2CFU/prms/rot70k.mol2', 
                 sep='\s+', header=None)

In [80]:
t1 = pd.read_csv('/home/mikhail/Projects/DOCK/libsaxs/complex_testing/benchmark/Real_new/5T1O/saxs_15_20.000000_51.000000_0.2', 
                 sep='\s+', header=None, skiprows=1)
t1.sort_values(1).iloc[:10000, :].to_csv('/home/mikhail/Projects/DOCK/libsaxs/complex_testing/benchmark/Real_new/5T1O/saxs_10000', 
                 sep='\t', header=None, index=None)